In [ ]:
ENV["LINES"] = 30
ENV["COLUMNS"] = 300

In [ ]:

using Pipe

In [ ]:
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end

In [4]:
using PyCall
@pyimport nltk

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2499: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [5]:
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [6]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [7]:
LL,word_index,indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_index|> typeof |> println
indexed_words|> typeof |> println

(50,268810)
Dict{String,BitArray{1}}
Array{String,1}


In [66]:
typealias Embedding Vector{Float64}
typealias Embeddings Matrix{Float64}
typealias Words Union(AbstractArray{ASCIIString,1},AbstractArray{String,1})
type RAE
    L::Matrix{Float64}
    word_index::Dict{String,BitVector}
    indexed_words::Vector{String}
    
    W_e::Matrix{Float64}
    W_d::Matrix{Float64}
   
end


function RAE(L::Matrix{Float64}, word_index::Dict{String,BitVector}, indexed_words::Vector{String})
    emb_width = size(L,1)
    
    W_e =0.01*randn(emb_width,emb_width*2+1) 
    #W_d = 0.01*randn(emb_width*2,emb_width+1)
    W_d = [pinv(W_e)[1:end-1,:] zeros(size(pinv(W_e),1)-1) ] #Cheat (Actually why can't I always do this to initialize?);
    RAE(L,word_index, indexed_words, W_e, W_d)
end

function eval_embedding(rae::RAE, input::String)
    if haskey(word_index, input)
        k = word_index[input]
    elseif haskey(word_index, lowercase(input))
        k = word_index[lowercase(input)]
    else
        k = word_index["*UNKNOWN*"]
        println("$input not found. Defaulting.")
    end
    rae.L*k
end

function eval_embeddings(rae::RAE, inputs::Words)
    ks= @pipe inputs |> map(ii -> word_index[ii], _) |> hcat(_...)
    rae.L*ks
end

function eval_embeddings(rae::RAE, c_is::Embeddings, c_js::Embeddings)
    @assert size(c_is)==size(c_js)
    bias_in = ones(1,size([c_is], 2))
    tanh(rae.W_e*[c_is;c_js;bias_in])
end



eval_embeddings (generic function with 2 methods)

In [57]:
find(word_index["*UNKNOWN*"])

1-element Array{Int64,1}:
 94906

In [63]:
lowercase("A")

"a"

In [9]:
function eval_scores(rae::RAE, c_is::Embeddings, c_js::Embeddings)
    pps=eval_embeddings(rae, c_is, c_js)
    c_ijs = [c_is;c_js]
    bias_in = ones(1,size(pps, 2))
    ĉ_ijs = tanh(rae.W_d*[pps;bias_in])
    
    sum((c_ijs-ĉ_ijs).^2,1)
end

#A better scoring function is given in SorcherRAE (eaquation 6)


eval_scores (generic function with 1 method)

In [10]:
function eval_to_tree(rr::RAE,sentence::String)
    eval_to_tree(rr, tokenize(sentence))
end

function eval_to_tree(rr::RAE, sentence::Words)
    tree = tuple(sentence...)
    cs = eval_embeddings(rr, sentence)
    score_total = 0.0
    while(size(cs,2)>1)
        c_is = cs[:, 1:end-1]
        c_js = cs[:, 2:end]
        
        pps = eval_embeddings(rr, c_is, c_js)
        scores = eval_scores(rr, c_is, c_js)
        im = indmax(scores)
        
        score_total+=scores[im]
         
        cs = [cs[:,1:im-1] pps[:,im] cs[:,im+2:end]]
        tree = tuple(tree[1:im-1]..., (tree[im], tree[im+1]), tree[im+2:end]...)
    end
    tree = tree[1] #The final step creates a tuple containing one element, as first and last parts are empty
    embedding = cs[:]
    tree, embedding, score_total
end


eval_to_tree (generic function with 2 methods)

In [ ]:
rr = RAE(LL,word_index,indexed_words);

sent = "the boy destroyed the house"
sent_toks = tokenize(sent)

tree, pp, score_total = eval_to_tree(rr,sent_toks)


In [15]:
using Optim

rae_outer = RAE(LL,word_index,indexed_words);

function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    W_d_len = length(rae.W_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1:W_e_len],W_e_shape)
    rae.W_d = reshape(θ[W_e_len+1:end],W_d_shape)
    
    rae
end

function pack(rae::RAE)
    [rae.W_e[:], rae.W_d[:]] 
end

function f(θ::Vector)
    rae = unpack(rae_outer, θ)
    
    ŷ = feedforward(rae, data_train)[1]
    error_rate(ŷ, lbls_train) 
end


#res = optimize(f, pack(rae_outer), method=:l_bfgs, show_trace = true, iterations = 10)
println(res)
rae_outer = unpack(rae_outer, res.minimum)

LoadError: res not defined
while loading In[15], in expression starting on line 30

In [33]:
using Iterators
@pyimport nltk.corpus as nltk_corpus


In [23]:
for (ii,sent) in enumerate(nltk_corpus.brown[:sents]())
    println(sent)
    if ii>10
        break
    end
end

{"The","Fulton","County","Grand","Jury","said","Friday","an","investigation","of","Atlanta's","recent","primary","election","produced","``","no","evidence","''","that","any","irregularities","took","place","."}
{"The","jury","further","said","in","term-end","presentments","that","the","City","Executive","Committee",",","which","had","over-all","charge","of","the","election",",","``","deserves","the","praise","and","thanks","of","the","City","of","Atlanta","''","for","the","manner","in","which","the","election","was","conducted","."}
{"The","September-October","term","jury","had","been","charged","by","Fulton","Superior","Court","Judge","Durwood","Pye","to","investigate","reports","of","possible","``","irregularities","''","in","the","hard-fought","primary","which","was","won","by","Mayor-nominate","Ivan","Allen","Jr.","."}
{"``","Only","a","relative","handful","of","such","reports","was","received","''",",","the","jury","said",",","``","considering","the","widespread","interest","in","

In [71]:
n_training = 1000
sents = @pipe nltk_corpus.brown[:sents]() |> take(_,n_training)  |> collect |> convert(Vector{Vector{String}},_)


1000-element Array{Array{String,1},1}:
 String["The","Fulton","County","Grand","Jury","said","Friday","an","investigation","of"  …  "``","no","evidence","''","that","any","irregularities","took","place","."]         
 String["The","jury","further","said","in","term-end","presentments","that","the","City"  …  "for","the","manner","in","which","the","election","was","conducted","."]           
 String["The","September-October","term","jury","had","been","charged","by","Fulton","Superior"  …  "primary","which","was","won","by","Mayor-nominate","Ivan","Allen","Jr.","."]
 String["``","Only","a","relative","handful","of","such","reports","was","received"  …  "of","voters","and","the","size","of","this","city","''","."]                            
 String["The","jury","said","it","did","find","that","many","of","Georgia's"  …  "``","are","outmoded","or","inadequate","and","often","ambiguous","''","."]                     
 String["It","recommended","that","Fulton","legislators","act","``","to

Haandful not found. Defaulting.


50-element Array{Float64,1}:
  0.0398781 
  0.048215  
  0.0176077 
 -0.0251486 
 -0.0627525 
  0.0251626 
  0.0117941 
  0.0613618 
  0.0182828 
  0.028989  
  0.0305115 
 -0.0174257 
  0.0267883 
  ⋮         
 -0.00159508
 -0.0682754 
 -0.0136059 
 -0.0243803 
 -0.0181979 
  0.00254263
 -0.0295249 
 -0.0390804 
 -0.0137563 
  0.0252652 
  0.0304082 
  0.0337324 

In [ ]:
#tree data in tree is not use, other than it's structure.
#((("the","house"),("destroyed",("the","boy")))  is equivalent to ((("",""),("",("",""))) 

function reconstruct(rae::RAE, pp::Embedding)
    bias_in = 1
    ĉ_ijs = tanh(rae.W_d*[pp;bias_in])
    ĉ_is = ĉ_ijs[1:end/2]
    ĉ_js = ĉ_ijs[end/2+1:end]
    ĉ_is, ĉ_js
end

function unfold(rae::RAE, tree::(String,String), pp::Embedding)
    ĉ_is, ĉ_js = reconstruct(rae, pp)
    [ĉ_is ĉ_js]
end


function unfold(rae::RAE, tree::(Any,String), pp::Embedding)
    p̂_is, ĉ_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    [ĉ_is ĉ_js]
end

function unfold(rae::RAE, tree::(String,Any), pp::Embedding)
    ĉ_is, p̂_js = reconstruct(rae, pp)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
    
end

function unfold(rae::RAE, tree::(Any,Any), pp::Embedding)
    p̂_is, p̂_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
end

    

In [ ]:
tree, pp, score_total = eval_to_tree(rr,"killer cows")
ĉs = unfold(rr,tree,pp)

show_bests(rr, ĉs)


In [ ]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] score] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


In [ ]:
function depth_inc(ele::(Int,String))
    (ele[1]+1,ele[2])
end

function unfold_struct(tree::(Any,Any))
    left_tree = unfold_struct(tree[1]) 
    left = @pipe left_tree |> map(depth_inc,_)
    right_tree = unfold_struct(tree[2]) 
    right = @pipe right_tree |> map(depth_inc,_)
    [left, right, (0,"")]
end

function unfold_struct(tree::(Any,String))
    left_tree = unfold_struct(tree[1]) 
    left = @pipe left_tree |> map(depth_inc,_)
    [left, (0,tree[2]), (0,"")]
end
function unfold_struct(tree::(String,Any))
    right_tree = unfold_struct(tree[2]) 
    right = @pipe right_tree |> map(depth_inc,_)
    [(0,tree[1]),right, (0,"")]
end
function unfold_struct(tree::(String,String))
    [(0,tree[1]), (0, tree[2]), (0,"")]
end

function print_tree(tree::(Any,Any))
    
    for (depth,word ) in unfold_struct(tree)
        println("\t"^depth, word)
    end
end